In [8]:
!pip install -q transformers datasets peft trl bitsandbytes accelerate scipy einops

In [19]:
from datasets import load_dataset

dataset = load_dataset("json", data_files=r"qa_format_quotes.jsonl")

In [20]:
dataset

DatasetDict({
    train: Dataset({
        features: ['prompt', 'completion'],
        num_rows: 2994
    })
})

In [26]:
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig
from dotenv import load_dotenv

load_dotenv()

model = "microsoft/phi-2"

bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_compute_dtype="bfloat16",
)

tokenizer = AutoTokenizer.from_pretrained(model, trust_remote_code=True)
model = AutoModelForCausalLM.from_pretrained(
    model,
    trust_remote_code=True,
    quantization_config=bnb_config,
    device_map="auto")

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [29]:
# LoRA Configurations
from peft import LoraConfig, get_peft_model

lora_config = LoraConfig(
    r=8,
    lora_alpha=32,
    target_modules=["q_proj", "v_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()

trainable params: 2,621,440 || all params: 2,782,305,280 || trainable%: 0.0942


In [30]:
# Training Arguments
from transformers import TrainingArguments

training_args = TrainingArguments(
    output_dir = ".\phi2_lora",
    per_device_train_batch_size=32,
    gradient_accumulation_steps=4,
    learning_rate=2e-4,
    num_train_epochs=2,
    logging_steps=25,
    report_to="none",
    save_strategy="epoch",
    warmup_steps=10
)

In [31]:
# Trainer
from trl import SFTTrainer

trainer = SFTTrainer(
    model=model,
    train_dataset=dataset["train"],
    args=training_args
)

Adding EOS to train dataset:   0%|          | 0/2994 [00:00<?, ? examples/s]

Tokenizing train dataset:   0%|          | 0/2994 [00:00<?, ? examples/s]

C:\Users\rajan\PycharmProjects\Tolkienizer\.venv\Lib\site-packages\trl\trainer\sft_trainer.py:767: UserWarning: Mismatch between tokenized prompt and the start of tokenized prompt+completion. This may be due to unexpected tokenizer behavior, whitespace issues, or special token handling. Verify that the tokenizer is processing text consistently.
  warnings.warn(


Truncating train dataset:   0%|          | 0/2994 [00:00<?, ? examples/s]

In [ ]:
trainer.train()

C:\Users\rajan\PycharmProjects\Tolkienizer\.venv\Lib\site-packages\torch\utils\data\dataloader.py:665: UserWarning: 'pin_memory' argument is set as true but no accelerator is found, then device pinned memory won't be used.
  warnings.warn(warn_msg)


In [ ]:
trainer.model.save_pretrained("./tokenizer_model/tokenizer_lora")
tokenizer.save_pretrained("./tokenizer_model/tokenizer_lora")